In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        return x

In [3]:
net = Network()

In [4]:
net

Network(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [5]:
def printnorm(self, input, output):
    print("Inside " + self.__class__.__name__ + ' forward')
    print('')
    print("input:", type(input), len(input))
    print("input[0]: ", type(input[0]), input[0].size())
    print('output: ', type(output))
    print("")
    print('input size: ', input[0].size())
    print("output size: ", output.data.size())
    print('output norm: ', output.data.norm())

In [6]:
net.conv1.register_forward_hook(printnorm)

In [7]:
x = torch.randn(1, 1, 28, 28)

In [8]:
output = net(x)

Inside Conv2d forward

input: <class 'tuple'> 1
input[0]:  <class 'torch.Tensor'> torch.Size([1, 1, 28, 28])
output:  <class 'torch.Tensor'>

input size:  torch.Size([1, 1, 28, 28])
output size:  torch.Size([1, 10, 24, 24])
output norm:  tensor(44.3613)


In [9]:
net.conv2.register_forward_hook(printnorm)

In [10]:
output = net(x)

Inside Conv2d forward

input: <class 'tuple'> 1
input[0]:  <class 'torch.Tensor'> torch.Size([1, 1, 28, 28])
output:  <class 'torch.Tensor'>

input size:  torch.Size([1, 1, 28, 28])
output size:  torch.Size([1, 10, 24, 24])
output norm:  tensor(44.3613)
Inside Conv2d forward

input: <class 'tuple'> 1
input[0]:  <class 'torch.Tensor'> torch.Size([1, 10, 12, 12])
output:  <class 'torch.Tensor'>

input size:  torch.Size([1, 10, 12, 12])
output size:  torch.Size([1, 20, 8, 8])
output norm:  tensor(11.8931)


In [11]:
def printgradnorm(self, grad_input, grad_output):
    print("Inside " + self.__class__.__name__ + ' backward')
    print('')
    print("grad_input:", type(grad_input), len(grad_input))
    print("grad_input[0]: ", type(grad_input[0]), grad_input[0].size())
    print("grad_input[1]: ", type(grad_input[1]), grad_input[1].size())
    print("grad_input[2]: ", type(grad_input[2]), grad_input[2].size())

    print('grad_output: ', type(grad_output), len(grad_output))
    print('grad_output[0]: ', type(grad_output[0]))
    print("")
    print('grad_input size: ', grad_input[0].size())
    print("grad_output size: ", grad_output[0].data.size())
    print('grad_output norm: ', grad_output[0].data.norm())

In [12]:
net.conv2.register_backward_hook(printgradnorm)

In [13]:
output = net(x)

Inside Conv2d forward

input: <class 'tuple'> 1
input[0]:  <class 'torch.Tensor'> torch.Size([1, 1, 28, 28])
output:  <class 'torch.Tensor'>

input size:  torch.Size([1, 1, 28, 28])
output size:  torch.Size([1, 10, 24, 24])
output norm:  tensor(44.3613)
Inside Conv2d forward

input: <class 'tuple'> 1
input[0]:  <class 'torch.Tensor'> torch.Size([1, 10, 12, 12])
output:  <class 'torch.Tensor'>

input size:  torch.Size([1, 10, 12, 12])
output size:  torch.Size([1, 20, 8, 8])
output norm:  tensor(11.8931)


In [14]:
target = torch.tensor([3], dtype=torch.long)
loss_fn = nn.CrossEntropyLoss()

In [15]:
err = loss_fn(output, target)

In [16]:
err.backward()

Inside Conv2d backward

grad_input: <class 'tuple'> 3
grad_input[0]:  <class 'torch.Tensor'> torch.Size([1, 10, 12, 12])
grad_input[1]:  <class 'torch.Tensor'> torch.Size([20, 10, 5, 5])
grad_input[2]:  <class 'torch.Tensor'> torch.Size([20])
grad_output:  <class 'tuple'> 1
grad_output[0]:  <class 'torch.Tensor'>

grad_input size:  torch.Size([1, 10, 12, 12])
grad_output size:  torch.Size([1, 20, 8, 8])
grad_output norm:  tensor(0.1975)
